<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/LWF_by_Alb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL

Cloning into 'Project_MLDL'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 292 (delta 42), reused 0 (delta 0), pack-reused 220
Receiving objects: 100% (292/292), 943.53 KiB | 7.93 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [0]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import time
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import copy
import torch.nn.functional as F

from PIL import Image
from Project_MLDL.CIFAR100_tError import CIFAR100_tError
from Project_MLDL.model_finetuning import ResNet18
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.models import resnet34
from tqdm import tqdm

# Garbage collector
gc.enable()

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Init at 10 because first train is on 10 classes
NUM_CLASSES = 10

# Used for the pseudorandom shuffle of the split
SEED = 12

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 2         # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70     # Total number of training epochs (iterations over dataset)
MILESTONES = [48, 62]  # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.2          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 50

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), # Randomly flip the image with probability of 0.5
                                      transforms.Pad(4), # Add padding
                                      transforms.RandomCrop(32),# Crops a random squares of the image
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))                                 
])

In [0]:
DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]  #Array of classes integer-encoded (?)
random.seed(SEED)
random.shuffle(lbls)

def make_data_labels(lbls):       #After shuffle, take first 10 classes, and remove the first 10 from the list passed as argument
    new_labels=[]
    for el in lbls[:10]:
        new_labels.append(el)
    lbls = lbls[10:]

    return lbls, new_labels

In [0]:
def distillation_loss(t, s, stud_nodes):
  prob_t = F.softmax(t, dim=1)
  log_prob_s = F.log_softmax(s[:, :stud_nodes], dim=1)
  dist_loss = -(prob_t*log_prob_s).sum(dim=1).mean()
  return dist_loss

class LWFNet(nn.Module):

  def __init__(self):
    super(LWFNet, self).__init__()
    
    self.feature_extractor = ResNet18()
    self.feature_extractor.linear = nn.Linear(512, 2048)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(2048, 100)
    
    self.cls_loss = nn.BCEWithLogitsLoss()
    self.dist_loss = nn.BCELoss()

    self.known_classes = 0
    self.num_classes = 100


  def forward(self, x):
    x = self.feature_extractor(x)
    #x = nn.functional.norm(x)
    x = self.relu(x)
    x = self.fc(x)
    return x


  def classify(self, images):
    _, labels = torch.max(torch.softmax(self.forward(images), dim=1), dim=1, keepdim=False)
    return labels  
  

  def update_representation(self, training_dataset, num_new_classes):

    self.known_classes += num_new_classes

    # per sicurezza
    classes = self.known_classes

    # Student nuova rete per classification
    # Teacher rete precedente per distillation
    teacher_model = copy.deepcopy(self)
    teacher_model.to(DEVICE)

    # Aumento output nodes
    self.fc = nn.Linear(2048, classes).to(DEVICE)

    train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

    net.train()

    parameters_to_optimize = net.parameters()
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=GAMMA, last_epoch=-1)
    cudnn.benchmark # Calling this optimizes runtime
    current_step = 0

    # Start iterating over the epochs
    for epoch in range(NUM_EPOCHS):
      print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))
    
      # Iterate over the dataset
      for images, labels in train_dataloader:
        # Bring data over the device of choice
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
          
        optimizer.zero_grad() # Zero-ing the gradients

        outputs = net(images)  # Forward pass to the network

        # One hot encoding labels for binary cross-entropy loss
        labels_onehot = nn.functional.one_hot(labels,classes)
        labels_onehot = labels_onehot.type_as(outputs)

        loss = self.cls_loss(outputs, labels_onehot)

        # Se non sono le prime 10 classi..
        if (self.known_classes // 10) > 1:
          dist_loss = distillation_loss(teacher_model(images), net(images), self.known_classes)
          loss += dist_loss

        loss.backward()  # backward pass: computes gradients
        optimizer.step() # update weights based on accumulated gradients
        current_step += 1

      scheduler.step()  # Step the scheduler
    return

In [0]:
net = LWFNet()
net = net.to(DEVICE)

NUM_GROUPS = 10

test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)

for i in range(0, NUM_GROUPS):
  print('\n', f'Training network on group {i+1} of {NUM_GROUPS}', '\n')

  # Every iteration the first 10 classes of the shuffled list of all unseen 
  # classes are selected to train the network and removed from the shuffled list
  lbls, new_labels = make_data_labels(lbls)

  # Training set is the train dataset of exactly 10 new classes
  train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)
  train_dataset.increment(new_labels, [j for j in range(0+i*10, 10+i*10)])

  # Evaluation set is the subset of the test dataset of the only 10 new classes
  # (for this reason -> "Not so fair" Validation)
  eval_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)
  eval_dataset.increment(new_labels, [j for j in range(0+i*10, 10+i*10)])    
    
  # Test dataset is the test dataset of the 10 new classes and all the previous
  test_dataset.increment(new_labels, [j for j in range(0+i*10, 10+i*10)])
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    
  net.update_representation(train_dataset, NUM_GROUPS)

  # Test on Test set
  net.train(False)
  running_corrects = 0
  for images,labels in test_dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    preds = net.classify(images)

    running_corrects += torch.sum(preds == labels.data).data.item()

    accuracy = running_corrects / float(len(test_dataset))
    
  print(f"Test Accuracy: {accuracy}")